## Anthropic

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

CLAUDE_API_KEY = os.environ.get('CLAUDE_API_KEY')

llm_config_claude35sonnet = [
    {
        "model": "claude-3-5-sonnet-20240620", 
        "api_key": CLAUDE_API_KEY, 
        "cache_seed": 41, 
        "temperature": 0,
        "api_type": "anthropic",
    }
]

## Google

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

GOOGLE_API_KEY = os.environ.get('GOOGLE_API_KEY')

llm_config_gemini15 = [
    {
        "model": "gemini-1.5-flash", 
        "api_key": GOOGLE_API_KEY, 
        "cache_seed": 41, 
        "temperature": 0,
        "api_type": "google",
    }
]

llm_config_gemini15Pro = [
    {
        "model": "gemini-1.5-pro", 
        "api_key": GOOGLE_API_KEY, 
        "cache_seed": 41, 
        "temperature": 0,
        "api_type": "google",
    }
]

## Local Tools

In [3]:
import logging
import configparser

def load_routers(routers_file='routers.csv'):
    """
    Loads router information from the .routers file.

    Args:
        routers_file (str): Path to the .routers file

    Returns:
        list: List of dictionaries containing router information
    """
    routers = []
    try:
        with open(routers_file, 'r') as f:
            for line in f:
                ip, username, password = line.strip().split(',')
                routers.append({'ip': ip, 'username': username, 'password': password})
        logging.info(f"Successfully loaded router information from {routers_file}")
        return routers
    except Exception as e:
        logging.error(f"Error loading router information: {e}")
    return []

def load_optics_cmds(optics_cmds_file='optics_cmds.txt'):
    """
    Reads the optics cmds file.

    Args:
        optics_cmds_file (str): Path to the optics_cmds_file file

    Returns:
        dict or None: Parsed optics_cmds if successful, None otherwise
    """
    try:
        with open(optics_cmds_file, 'r') as f:
            optics_cmds_file = f.read()
        #parsed_optics_cmds_file = configparser.parse_config(optics_cmds_file)
        logging.info(f"Successfully loaded optics_cmds_file from {optics_cmds_file}")
        return optics_cmds_file
    except Exception as e:
        logging.error(f"Error loading optics_cmds_file: {e}")
    return None

def load_optics_best(optics_best_file='optics_best.txt'):
    """
    Reads the optics_best file.

    Args:
        optics_best_file (str): Path to the optics_best_file file

    Returns:
        dict or None: Parsed optics_best if successful, None otherwise
    """
    try:
        with open(optics_best_file, 'r') as f:
            optics_best_file = f.read()
        #parsed_optics_best_file = configparser.parse_config(optics_best_file)
        logging.info(f"Successfully loaded optics_best_file from {optics_best_file}")
        return optics_best_file
    except Exception as e:
        logging.error(f"Error loading optics_best_file: {e}")
    return None

## Autogen

In [6]:
from autogen import ConversableAgent
from autogen import GroupChat
from autogen import GroupChatManager
from autogen import UserProxyAgent
from autogen import AssistantAgent
from autogen.coding import LocalCommandLineCodeExecutor
from autogen import register_function

llm_config_gemini = {
        "config_list": llm_config_gemini15,
    }

llm_config_geminiPro = {
        "config_list": llm_config_gemini15Pro,
    }

llm_config_claude = {
        "config_list": llm_config_claude35sonnet,
    }

def state_transition(last_speaker, groupchat):
        
    messages = groupchat.messages

    if len(messages) == 1:
            # First message handling
            print("First message detected...")
            return Netengineer

    last_message = messages[-1]

    if "tool_calls" in last_message:
        # Add code for tool call handling here if needed
            print("Tool call detected...")
            pass
        
    elif (not "content" in last_message):
        raise Exception("Message does not have `content` key")
    
    last_message_content = last_message["content"].strip()
    if not last_message_content:
        # Add code for blank message handlig here if needed
        print("Blank message detected...")
        pass
    
    print("Last speaker detected:", last_message["name"])

    coder_agents = {"Coder" : Coder, "Netengineer" : Netengineer}
    
    code_block_identifiers = ["```python", "```bash"]
    
    if last_speaker in coder_agents.values() and any(code_block_id in last_message_content for code_block_id in code_block_identifiers):
        print("Executing code written by", last_message["name"])
        return user_proxy
    
    elif last_speaker == user_proxy:
        if "exitcode: 1" in last_message_content:
            last_coder_agent_idx = -2
            while last_coder_agent_idx >= -len(messages):
                if messages[last_coder_agent_idx]["name"] in coder_agents:
                    break
                last_coder_agent_idx -= 1
            last_coder_agent_name = messages[last_coder_agent_idx]["name"]
            print("Error detected in code written by", last_coder_agent_name)
            return coder_agents[last_coder_agent_name]
        else:
                return "auto"
    
    else:
            return "auto"

Coder = ConversableAgent(
    name="Coder",
    system_message="""You are a Python developer. You write Python scripts to collect data from networking devices. 
     """,
    llm_config=llm_config_gemini,
)

Netengineer = ConversableAgent(
    name="Netengineer",
    system_message="""You are a network engineer. You review and enhance Python scripts. You execute and fix script errors.
    You need to parse the command instructions to extract the correct commands to run on a device, and update the script accordingly.
    You run scripts to collect network device data and provide recommendations to data analysts.
     """,
    llm_config=llm_config_gemini,
)

Analyst = AssistantAgent(
    name="Analyst",
    system_message="You are a data analyst. You take the network outputs and apply best practices to generate summaries and recommendations",
    llm_config=llm_config_gemini # llm_config_claude,
)

Reporter = AssistantAgent(
    name="Reporter",
    system_message="You are a networking reporter. You write network report based on recommendations",
    llm_config=llm_config_gemini,
)

user_proxy = UserProxyAgent(
    name = "user_proxy",
    system_message = "human admin",
    human_input_mode = "ALWAYS",
    max_consecutive_auto_reply=10,
    is_termination_msg=lambda x: x.get("content", "") and x.get(
        "content", "").rstrip().endswith("TERMINATE"),
    code_execution_config={
        # the executor to run the generated code
        "executor": LocalCommandLineCodeExecutor(work_dir="tmp_dir"),
    },
    description="""I **ALWAYS** speak **immediately** after `Coder`, `Unittester` and `Systester`."""
)



group_chat = GroupChat(
    agents = [user_proxy, Coder, Netengineer, Analyst, Reporter],
    messages = [],
    max_round = 6,
    send_introductions= True,
    speaker_selection_method = state_transition,
)

group_chat_manager = GroupChatManager(
    groupchat = group_chat,
    llm_config = llm_config_gemini,
)

chat_result = user_proxy.initiate_chat(
    group_chat_manager,
    message="""key steps: collect optics controller data, analyze the data, apply best practices and generate summaries and recommendations. 
    Devices are Cisco IOS-XR routers (provided by routers.csv file; 
    which is a device file, and the order of data is described by the first row of the file,
    hostname,ip_address,username,password,device_type).
    The type of data to be collected are specified by these two commands:
    show controller optics 0/0/0/[x]
    show controller CoherentDSP 0/0/0/[x]
    [x] is a list. Collect output for all instances of x. Here is an instruction on how to determine [x] for each device:
    Run this command: show running | include controller Optics
    all instances of [x] are listed in the output.
    Command output parsing instructions are provided by optics_parse.txt.
    The best practices are provided by optics_best.txt file. 
    The report should summarize recommendations for all the devices provided.
    """,
    summary_method="reflection_with_llm",
    max_turns=10,
    clear_history=True,
)

user_proxy (to chat_manager):

key steps: collect optics controller data, analyze the data, apply best practices and generate summaries and recommendations. 
    Devices are Cisco IOS-XR routers (provided by routers.csv file; 
    which is a device file, and the order of data is described by the first row of the file,
    hostname,ip_address,username,password,device_type).
    The type of data to be collected are specified by these two commands:
    show controller optics 0/0/0/[x]
    show controller CoherentDSP 0/0/0/[x]
    [x] is a list. Collect output for all instances of x. Here is an instruction on how to determine [x] for each device:
    Run this command: show running | include controller Optics
    all instances of [x] are listed in the output.
    Command output parsing instructions are provided by optics_parse.txt.
    The best practices are provided by optics_best.txt file. 
    The report should summarize recommendations for all the devices provided.
    

---------------